# LlamaIndex - 面向 LLM 的数据框架教程 (RAG 重点)

欢迎来到 LlamaIndex 教程！LlamaIndex (曾用名 GPT Index) 是一个开源的数据框架，专门用于将自定义数据源连接到大型语言模型 (LLM)，从而构建强大的**检索增强生成 (Retrieval-Augmented Generation, RAG)** 应用程序。

与 LangChain 类似，LlamaIndex 也旨在简化 LLM 应用开发，但它**特别专注于数据的摄入 (Ingestion)、索引 (Indexing) 和查询 (Querying)** 过程，为构建高效、准确的 RAG 系统提供了丰富的高级工具和策略。

**为什么使用 LlamaIndex？**

1.  **强大的 RAG 功能**: 提供了从数据加载、解析、索引到复杂查询策略的全套 RAG 工具链。
2.  **多样化的数据连接器 (Readers/Loaders)**: 支持从各种来源（PDF, API, 数据库, Notion, Slack 等）加载数据。
3.  **灵活的索引结构**: 支持多种索引类型（向量索引、列表索引、关键词表索引、树索引等），适用于不同场景。
4.  **高级查询引擎**: 提供了超越简单相似性搜索的查询策略，如多步查询、联合查询、带综合的查询等。
5.  **与 LLM 和 Embedding 模型解耦**: 可以灵活搭配不同的 LLM 和 Embedding 模型。
6.  **可观测性与评估**: 集成了调试和评估 RAG 应用的工具。
7.  **与 LangChain 可集成**: 两者可以相互配合使用。

**核心概念概览:**
*   **Documents**: 数据的容器，可以是文本文件、PDF 或其他来源的数据。
*   **Nodes**: 文档被解析成的原子数据单元（通常是文本块），包含文本和元数据。
*   **Readers/Loaders**: 用于从数据源加载数据并创建 `Document` 对象。
*   **Indexes**: 根据 Nodes 构建的数据结构，用于高效检索。
*   **Embeddings**: 将文本转换为向量表示的模型。
*   **Retrievers**: 从索引中根据查询检索相关 Nodes 的组件。
*   **Node Postprocessors**: 对检索到的 Nodes 进行重新排序或过滤。
*   **Response Synthesizers**: 根据检索到的上下文和原始查询生成最终答案。
*   **Query Engines**: 封装了从查询到生成响应的端到端逻辑 (Retrieval -> Postprocessing -> Synthesis)。
*   **Chat Engines**: 用于构建基于索引数据的对话式应用。

**本教程将涵盖 LlamaIndex 的核心 RAG 工作流程：**

1.  安装与设置 (API Keys)
2.  数据加载 (SimpleDirectoryReader)
3.  文档解析与节点创建 (NodeParser)
4.  服务上下文 (ServiceContext) 与模型配置 (LLM, Embeddings)
5.  构建索引 (VectorStoreIndex)
6.  创建查询引擎 (Query Engine)
7.  进行查询与获取响应
8.  (简介) 创建聊天引擎 (Chat Engine)
9.  (简介) 保存与加载索引

## 1. 安装与设置

安装 LlamaIndex 核心库。根据你使用的 LLM 和 Embedding 模型，可能需要安装额外的库 (如 `openai`, `transformers`, `sentence-transformers`)。

```bash
pip install llama-index

# 如果使用 OpenAI 模型 (推荐开始)
pip install openai python-dotenv

# 如果使用 HuggingFace 模型 (可选)
# pip install transformers torch sentence-transformers accelerate # accelerate for faster loading
```

**设置 API Keys**: 与 LangChain 类似，将你的 OpenAI API Key (或其他 LLM Key) 存储在环境变量或 `.env` 文件中。

In [ ]:
import llama_index
import os
from dotenv import load_dotenv
import logging
import sys

# 配置日志记录，方便观察 LlamaIndex 内部流程 (可选)
# logging.basicConfig(stream=sys.stdout, level=logging.INFO) # INFO level
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# 尝试加载 .env 文件
load_success = load_dotenv() 
print(f"LlamaIndex version: {llama_index.__version__}")
print(f".env file loaded: {load_success}")

# 检查 OpenAI API Key
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print("OpenAI API Key found.")
    openai_available_llama = True
else:
    print("OpenAI API Key not found. OpenAI related examples will be skipped.")
    openai_available_llama = False

# 可以手动设置 Key，但不推荐
# import openai
# openai.api_key = "sk-..."

## 2. 数据加载 (SimpleDirectoryReader)

LlamaIndex 提供了多种 `Reader` (或称 `Loader`) 来加载不同来源的数据。`SimpleDirectoryReader` 是最常用的之一，它可以加载一个目录下所有支持的文件 (如 .txt, .pdf, .docx)。

In [ ]:
from llama_index import SimpleDirectoryReader

print("--- Data Loading --- ")

# 1. 创建一个包含示例文件的目录
data_dir = "llamaindex_data"
os.makedirs(data_dir, exist_ok=True)

file1_path = os.path.join(data_dir, "file1.txt")
file2_path = os.path.join(data_dir, "file2.md")

with open(file1_path, "w") as f:
    f.write("The first document discusses apples and oranges. Apples are red or green, oranges are orange.")
with open(file2_path, "w") as f:
    f.write("# Fruits Summary\n\nBananas are yellow and curved. Grapes grow in bunches.")
print(f"Created sample files in '{data_dir}' directory.")

# 2. 使用 SimpleDirectoryReader 加载数据
try:
    reader = SimpleDirectoryReader(data_dir)
    documents = reader.load_data()
    print(f"\nLoaded {len(documents)} documents.")
    
    # Document 对象包含 text 和 metadata
    print("\nContent of the first document:")
    print(f"  Text: {documents[0].text}")
    print(f"  Metadata: {documents[0].metadata}")
    
    print("\nContent of the second document:")
    print(f"  Text: {documents[1].text}")
    print(f"  Metadata: {documents[1].metadata}")
    
    documents_available = True

except Exception as e:
    print(f"Error loading documents: {e}")
    documents_available = False

## 3. 文档解析与节点创建 (NodeParser)

加载的 `Document` 对象通常需要被分割成更小的 `Node` 对象（文本块），以便于 Embedding 和检索。LlamaIndex 提供了 `NodeParser` (例如 `SentenceSplitter`) 来完成这个任务。

In [ ]:
from llama_index.node_parser import SimpleNodeParser, SentenceSplitter
from llama_index.schema import Document # Import Document schema if needed for type hints

print("\n--- Node Parsing --- ")
nodes = []
if documents_available:
    # 使用 SentenceSplitter (更推荐，考虑句子边界)
    # chunk_size: 每个块的目标大小
    # chunk_overlap: 块之间的重叠
    node_parser = SentenceSplitter(chunk_size=64, chunk_overlap=10)
    
    # 从 Documents 生成 Nodes
    nodes = node_parser.get_nodes_from_documents(documents)
    
    print(f"Split documents into {len(nodes)} Nodes.")
    print("\nExample Nodes:")
    for i, node in enumerate(nodes[:3]): # Display first few nodes
        print(f"--- Node {i+1} ---")
        print(f"  Text: {node.get_content().replace('\n', ' ')}") # node.text is deprecated, use get_content()
        print(f"  Node ID: {node.node_id}")
        print(f"  Metadata: {node.metadata}") # Metadata includes original filename
        # print(f"  Relationships: {node.relationships}") # Shows links to other nodes/doc
else:
    print("Documents not loaded, skipping node parsing.")

## 4. 服务上下文 (ServiceContext) 与模型配置

**（注意：在 LlamaIndex 的较新版本 (>=0.10) 中，`ServiceContext` 已被弃用，推荐直接将 LLM 和 Embedding 模型传递给索引或查询引擎，或者使用全局设置 `Settings`。）**

为了演示配置，我们将使用新的 `Settings` 方法。

`Settings` 用于全局配置 LlamaIndex 使用的 LLM、Embedding 模型、NodeParser、CallbackManager 等。

In [ ]:
from llama_index.llms import OpenAI
from llama_index.embeddings import OpenAIEmbedding
# from llama_index.llms import HuggingFaceLLM
# from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import ServiceContext, Settings # Import Settings
import torch # Check torch availability for potential HuggingFace models

print("\n--- Configuring Models (using Settings) ---")

# --- 配置 LLM --- 
llm = None
if openai_available_llama:
    try:
        # 使用 OpenAI GPT-3.5 Turbo (默认)
        llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
        print("OpenAI LLM (gpt-3.5-turbo) configured.")
    except Exception as e:
        print(f"Error configuring OpenAI LLM: {e}")
# else:
#     # 尝试配置 Hugging Face LLM (如果需要且已安装)
#     try:
#         # 注意: 可能需要登录 Hugging Face Hub (huggingface-cli login)
#         # 并且需要安装 accelerate: pip install accelerate
#         # 选择一个合适的模型, e.g., "google/flan-t5-small"
#         # llm = HuggingFaceLLM(model_name="google/flan-t5-small", device_map="auto")
#         # print("HuggingFace LLM configured.")
#         pass # Keep commented out for simplicity
#     except Exception as e:
#         print(f"Could not configure HuggingFace LLM: {e}")

# --- 配置 Embedding 模型 --- 
embed_model = None
if openai_available_llama:
    try:
        embed_model = OpenAIEmbedding()
        print("OpenAI Embedding model configured.")
    except Exception as e:
        print(f"Error configuring OpenAI Embedding: {e}")
# else:
#     # 尝试配置 Hugging Face Embedding 模型
#     try:
#         # model_name = "sentence-transformers/all-MiniLM-L6-v2"
#         # embed_model = HuggingFaceEmbedding(model_name=model_name)
#         # print(f"HuggingFace Embedding model '{model_name}' configured.")
        pass # Keep commented out
#     except Exception as e:
#         print(f"Could not configure HuggingFace Embedding: {e}")

# --- 全局设置 (新方法 >= 0.10) ---
if llm: Settings.llm = llm
if embed_model: Settings.embed_model = embed_model
if 'node_parser' in locals() and node_parser: Settings.node_parser = node_parser
# Settings.chunk_size = 512 # Can also set chunk size globally

print("\nGlobal Settings configured (if models were available):")
print(f"  LLM: {Settings.llm}")
print(f"  Embed Model: {Settings.embed_model}")

# --- 旧方法: ServiceContext (了解即可) ---
# service_context = ServiceContext.from_defaults(
#     llm=llm, 
#     embed_model=embed_model, 
#     node_parser=node_parser # Can pass parser here too
# )

## 5. 构建索引 (VectorStoreIndex)

将处理好的 `Nodes` 转换为索引，以便快速检索。
`VectorStoreIndex` 是最常用的索引类型，它会为每个 Node 生成 embedding 向量，并存储在一个向量数据库中 (默认是内存中的简单实现，也可以配置为 FAISS, Chroma, Pinecone 等)。

In [ ]:
from llama_index import VectorStoreIndex

print("\n--- Building Index --- ")
index = None
# 确保有 nodes 并且 embedding 模型已配置
if nodes and Settings.embed_model:
    try:
        # 从 Nodes 构建索引，LlamaIndex 会使用 Settings 中配置的模型
        index = VectorStoreIndex(nodes)
        # 或者使用旧方法: index = VectorStoreIndex.from_documents(documents, service_context=service_context)
        print("VectorStoreIndex built successfully.")
        
    except Exception as e:
        print(f"Error building index: {e}")
        print("This might be due to issues with the embedding model or data.")
else:
    print("Skipping index building (Nodes or Embed model unavailable).")

## 6. 创建查询引擎 (Query Engine)

查询引擎封装了从索引进行查询并生成响应的逻辑。
`index.as_query_engine()` 是创建查询引擎的最简单方式。

In [ ]:
print("\n--- Creating Query Engine --- ")
query_engine = None
if index and Settings.llm: # Need index and LLM
    try:
        # similarity_top_k 控制检索多少个最相关的节点
        query_engine = index.as_query_engine(similarity_top_k=2)
        print("Query engine created.")
    except Exception as e:
        print(f"Error creating query engine: {e}")
else:
    print("Skipping query engine creation (Index or LLM unavailable).")

## 7. 进行查询与获取响应

使用查询引擎的 `.query()` 方法。

In [ ]:
print("\n--- Querying --- ")

if query_engine:
    query1 = "What colors can apples be?"
    print(f"Query 1: {query1}")
    try:
        response1 = query_engine.query(query1)
        print(f"Response 1: {response1}")
        # 查看检索到的源节点 (用于调试或引用)
        print("\nSource Nodes for Response 1:")
        for node in response1.source_nodes:
            print(f"  - Score: {node.score:.4f}, Text: {node.node.get_content().replace('\n',' ')}")

    except Exception as e:
        print(f"Error during query 1: {e}")
        
    query2 = "Tell me about bananas."
    print(f"\nQuery 2: {query2}")
    try:
        response2 = query_engine.query(query2)
        print(f"Response 2: {response2}")
        print("\nSource Nodes for Response 2:")
        for node in response2.source_nodes:
             print(f"  - Score: {node.score:.4f}, Text: {node.node.get_content().replace('\n',' ')}")

    except Exception as e:
        print(f"Error during query 2: {e}")
else:
    print("Query engine not available, skipping querying example.")

## 8. (简介) 创建聊天引擎 (Chat Engine)

如果你想构建一个基于索引数据的对话机器人（可以记住之前的对话内容），可以使用 `index.as_chat_engine()`。

**模式**: 
*   `condense_question`: 先将对话历史和新问题压缩成一个独立的问题，然后查询索引。
*   `context`: 在每轮对话中都检索上下文，并将历史记录和新检索到的上下文一起提供给 LLM。
*   `react`: 使用 ReAct 框架让 LLM 决定是查询索引还是直接回答。

```python
# if index and Settings.llm:
#     chat_engine = index.as_chat_engine(
#         chat_mode="condense_question", 
#         verbose=True
#     )
    
#     response_chat1 = chat_engine.chat("What fruits were discussed?")
#     print(response_chat1)
    
#     response_chat2 = chat_engine.chat("Tell me more about the red ones.") # Should use context
#     print(response_chat2)
    
#     # chat_engine.reset() # Reset chat history
```

## 9. (简介) 保存与加载索引

构建索引（特别是包含 embedding）可能比较耗时。LlamaIndex 允许你将索引的存储（包括向量存储）持久化到磁盘，以便后续快速加载。

```python
# from llama_index import StorageContext, load_index_from_storage

# # --- 保存 --- 
# index_persist_dir = "./saved_llama_index"
# if index:
#     index.storage_context.persist(persist_dir=index_persist_dir)
#     print(f"Index saved to {index_persist_dir}")

# # --- 加载 --- 
# # 需要确保 Settings (LLM, Embed model) 仍然配置正确
# try:
#     storage_context = StorageContext.from_defaults(persist_dir=index_persist_dir)
#     loaded_index = load_index_from_storage(storage_context)
#     print(f"Index loaded successfully from {index_persist_dir}")
#     # loaded_query_engine = loaded_index.as_query_engine()
#     # response = loaded_query_engine.query("What are apples like?")
#     # print(response)
# except FileNotFoundError:
#     print(f"Index directory {index_persist_dir} not found.")
# except Exception as e:
#      print(f"Error loading index: {e}")
```

## 总结

LlamaIndex 是一个强大的数据框架，专门用于构建基于 LLM 的 RAG 应用程序。它提供了灵活的数据加载、解析、索引和查询机制。

**关键要点：**
*   核心流程: **Load -> Parse (Nodes) -> Embed -> Index -> Query**。
*   `SimpleDirectoryReader` 用于方便地加载本地文件。
*   `SentenceSplitter` (或类似 NodeParser) 用于将文档分割成 Nodes。
*   通过 `Settings` (或旧的 `ServiceContext`) 配置 LLM 和 Embedding 模型。
*   `VectorStoreIndex` 是最常用的索引类型，用于高效的相似性搜索。
*   `index.as_query_engine()` 创建用于问答的查询引擎。
*   `index.as_chat_engine()` 创建用于对话的应用。
*   支持索引的持久化存储和加载。

对于需要将外部知识库或私有数据与 LLM 结合以提供更准确、更有依据的回答的应用场景，LlamaIndex 是一个非常有价值的工具。它与 LangChain 可以互补使用，LangChain 更侧重于链和 Agent 的编排，而 LlamaIndex 在 RAG 的数据处理和查询方面更深入。

In [ ]:
# --- Final Cleanup --- 
import shutil
if 'data_dir' in locals() and os.path.exists(data_dir):
    try:
        shutil.rmtree(data_dir)
        print(f"Cleaned up directory: {data_dir}")
    except Exception as e:
        print(f"Error cleaning up directory {data_dir}: {e}")

# index_persist_dir = "./saved_llama_index"
# if os.path.exists(index_persist_dir):
#     try:
#         shutil.rmtree(index_persist_dir)
#         print(f"Cleaned up directory: {index_persist_dir}")
#     except Exception as e:
#          print(f"Error cleaning up directory {index_persist_dir}: {e}")